In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 📈 Real vs. Nominal GDP Growth: The Base-Year Method

When comparing GDP over time, we need to distinguish between:

* **Nominal GDP:** The value of goods and services produced using **current prices**. It can increase due to either higher production *or* higher prices.
* **Real GDP:** The value of goods and services produced using **constant prices from a specific base year**. It isolates changes in the *quantity* of production by removing the effect of price changes.

This notebook demonstrates how to calculate Real GDP and its growth rate using the **fixed base-year** method for a simple two-good economy (e.g., Apples and Bananas).

# 🧮 Calculation: Base-Year Method

Let $P_{i,t}$ and $Q_{i,t}$ be the price and quantity of good $i$ in year $t$.

1.  **Choose a Base Year:** We select one year (e.g., Year 1 in this simulation) whose prices will be used for all calculations of Real GDP.
2.  **Calculate Nominal GDP:** Calculate the value of production in each year using that year's *own* prices.
    * Nominal GDP$_{Y1} = P_{apples, Y1} Q_{apples, Y1} + P_{bananas, Y1} Q_{bananas, Y1}$
    * Nominal GDP$_{Y2} = P_{apples, Y2} Q_{apples, Y2} + P_{bananas, Y2} Q_{bananas, Y2}$
3.  **Calculate Real GDP:** Calculate the value of production in each year using the *base year's* prices (Year 1 prices in this example).
    * Real GDP$_{Y1}$ (Base=Y1) = $P_{apples, Y1} Q_{apples, Y1} + P_{bananas, Y1} Q_{bananas, Y1}$  *(Note: Real GDP in the base year equals Nominal GDP in the base year)*
    * Real GDP$_{Y2}$ (Base=Y1) = $P_{apples, Y1} Q_{apples, Y2} + P_{bananas, Y1} Q_{bananas, Y2}$
4.  **Calculate Real GDP Growth Rate:** Find the percentage change between the Real GDP values.
    $$ g_{\text{Real}} = \frac{\text{Real GDP}_{Y2} - \text{Real GDP}_{Y1}}{\text{Real GDP}_{Y1}} \times 100\% $$

**GDP Deflator:** This method also allows us to calculate an implicit price index, the GDP Deflator, for Year 2 (relative to the base year Y1=100):
$$ \text{GDP Deflator}_{Y2} = \frac{\text{Nominal GDP}_{Y2}}{\text{Real GDP}_{Y2}} \times 100 $$
The inflation rate between Y1 and Y2 is approximately the percentage change in the GDP deflator.

*(Note: This simulation assumes prices are constant between Year 1 and Year 2 as set by the sliders, focusing only on quantity changes. A more complex example would allow prices to change too, making the distinction between Nominal and Real GDP more apparent).*

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def real_gdp_base_year_calc(
    q_apples_y1=100, q_bananas_y1=80,  # Quantities in Year 1 (Base Year)
    q_apples_y2=120, q_bananas_y2=100, # Quantities in Year 2
    p_apples_base=2.0, p_bananas_base=1.5 # Prices in Year 1 (Base Year)
    # Assume prices are constant in this simple version for base-year calculation
    ):
    """
    Calculates Nominal and Real GDP (using Year 1 as base) and Real GDP growth.

    Args:
        q_apples_y1, q_bananas_y1: Quantities of apples and bananas in Year 1.
        q_apples_y2, q_bananas_y2: Quantities of apples and bananas in Year 2.
        p_apples_base, p_bananas_base: Prices of apples and bananas in Year 1 (Base Year).
    """
    # --- Calculations ---
    # Assume prices are constant = base year prices for this simple example
    p_apples_y1 = p_apples_base
    p_bananas_y1 = p_bananas_base
    p_apples_y2 = p_apples_base # In this version, prices don't change
    p_bananas_y2 = p_bananas_base # To make Real=Nominal growth clear

    # Nominal GDP (using current quantities and current prices)
    nominal_gdp_y1 = (q_apples_y1 * p_apples_y1) + (q_bananas_y1 * p_bananas_y1)
    nominal_gdp_y2 = (q_apples_y2 * p_apples_y2) + (q_bananas_y2 * p_bananas_y2)

    # Real GDP (using current quantities and BASE YEAR prices)
    real_gdp_y1 = (q_apples_y1 * p_apples_base) + (q_bananas_y1 * p_bananas_base) # Real=Nominal in Base Year
    real_gdp_y2 = (q_apples_y2 * p_apples_base) + (q_bananas_y2 * p_bananas_base)

    # Real GDP Growth Rate
    if real_gdp_y1 != 0:
        real_growth_rate = (real_gdp_y2 - real_gdp_y1) / real_gdp_y1 * 100
    else:
        real_growth_rate = np.nan

    # GDP Deflator (relative to base year = 100)
    if real_gdp_y2 != 0:
        gdp_deflator_y1 = (nominal_gdp_y1 / real_gdp_y1) * 100 if real_gdp_y1 != 0 else np.nan
        gdp_deflator_y2 = (nominal_gdp_y2 / real_gdp_y2) * 100
        # Inflation = % change in deflator
        if not (np.isnan(gdp_deflator_y1) or gdp_deflator_y1 == 0):
             inflation_rate = (gdp_deflator_y2 / gdp_deflator_y1 - 1) * 100
        else:
             inflation_rate = np.nan
    else:
        gdp_deflator_y1 = np.nan
        gdp_deflator_y2 = np.nan
        inflation_rate = np.nan


    # --- Plotting ---
    years = ['Year 1 (Base)', 'Year 2']
    real_gdp_values = [real_gdp_y1, real_gdp_y2]
    nominal_gdp_values = [nominal_gdp_y1, nominal_gdp_y2]

    fig, ax = plt.subplots(figsize=(8, 5))
    bar_width = 0.35
    index = np.arange(len(years))

    bars1 = ax.bar(index - bar_width/2, nominal_gdp_values, bar_width, label='Nominal GDP', color='#ff7f0e', alpha=0.8, edgecolor='black')
    bars2 = ax.bar(index + bar_width/2, real_gdp_values, bar_width, label='Real GDP (Base Yr Prices)', color='#1f77b4', alpha=0.8, edgecolor='black')

    ax.set_title("Nominal vs. Real GDP (Base-Year Method)")
    ax.set_ylabel("GDP ($)")
    ax.set_xticks(index)
    ax.set_xticklabels(years)
    ax.legend()
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    # Add bar labels
    ax.bar_label(bars1, fmt='$%.0f', padding=3, fontsize=9)
    ax.bar_label(bars2, fmt='$%.0f', padding=3, fontsize=9)

    plt.tight_layout()
    plt.show()

    # --- Display Results ---
    results_md = f"""
    ### 🧮 Real GDP Calculation (Base Year = Year 1):

    **Inputs:**
    | Item         | Year 1 Qty | Year 2 Qty | Base Price (Yr 1) |
    |--------------|------------|------------|-------------------|
    | Apples       | {q_apples_y1}    | {q_apples_y2}    | ${p_apples_base:.2f}       |
    | Bananas      | {q_bananas_y1}    | {q_bananas_y2}    | ${p_bananas_base:.2f}       |

    **Calculations:**
    * **Nominal GDP Year 1:** ${nominal_gdp_y1:,.2f}$
    * **Nominal GDP Year 2:** ${nominal_gdp_y2:,.2f}$ *(Using Year 2 Qty & Assumed Yr 2 Prices = Base Prices)*
    * **Real GDP Year 1 (Base=Y1):** ${real_gdp_y1:,.2f}$
    * **Real GDP Year 2 (Base=Y1):** ${real_gdp_y2:,.2f}$
    * **Real GDP Growth Rate:** **{real_growth_rate:.2f}%**
    * **GDP Deflator Year 1:** {gdp_deflator_y1:.1f} *(Base Year = 100)*
    * **GDP Deflator Year 2:** {gdp_deflator_y2:.1f}
    * **Inflation Rate (Deflator % Chg):** {inflation_rate:.2f}% *(Note: Inflation is 0% here as prices were held constant)*

    *Real GDP isolates quantity changes by using fixed base-year prices.*
    """
    display(Markdown(results_md))


# --- Interactive Sliders ---
style = {'description_width': '150px'} # Wider description
layout = Layout(width='95%')

interact(
    real_gdp_base_year_calc,
    q_apples_y1=IntSlider(value=100, min=0, max=200, step=10, description='Qty Apples (Yr 1):', style=style, layout=layout),
    q_bananas_y1=IntSlider(value=80, min=0, max=200, step=10, description='Qty Bananas (Yr 1):', style=style, layout=layout),
    q_apples_y2=IntSlider(value=120, min=0, max=200, step=10, description='Qty Apples (Yr 2):', style=style, layout=layout),
    q_bananas_y2=IntSlider(value=100, min=0, max=200, step=10, description='Qty Bananas (Yr 2):', style=style, layout=layout),
    p_apples_base=FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1, description='Price Apples (Base Yr):', style=style, layout=layout, readout_format='.2f'),
    p_bananas_base=FloatSlider(value=1.5, min=0.5, max=5.0, step=0.1, description='Price Bananas (Base Yr):', style=style, layout=layout, readout_format='.2f')
);


interactive(children=(IntSlider(value=100, description='Qty Apples (Yr 1):', layout=Layout(width='95%'), max=2…